In [57]:
import pandas as pd
import numpy as np
import re
import glob
from pathlib import Path
from datetime import datetime
from datetime import timedelta

In [58]:
ts = datetime.now().strftime('%Y%m%d')

In [59]:
df = pd.concat(pd.read_csv(f) for f in glob.glob(f"output/clean_csv/*.csv"))
df

,name,link_product,price,odometer,transmissions,fuel_type,car_type,update_date,dealership,year_made,brand
0,Mustang,/used-cars/for-sale/ford/mustang/2017/fm-my17/...,33990,"180,001 - 200,000",Automatic,Petrol,Coupe,20251105,brisbaneautogroup,2017,Ford
1,Fiesta,/used-cars/for-sale/ford/fiesta/2015/ambiente/...,6990,"180,001 - 200,000",Manual,Petrol,Hatchback,20251105,brisbaneautogroup,2015,Ford
2,2,/used-cars/for-sale/mazda/2/2011/neo/de-series...,7490,"160,001 - 180,000",Automatic,Petrol,Hatchback,20251105,brisbaneautogroup,2011,Mazda
3,Micra,/used-cars/for-sale/nissan/micra/2016/st/k13-m...,7990,"120,001 - 140,000",Automatic,Petrol,Hatchback,20251105,brisbaneautogroup,2016,Nissan
4,3,/used-cars/for-sale/mazda/3/2014/sp25-astina/b...,9990,"140,001 - 160,000",Automatic,Petrol,Hatchback,20251105,brisbaneautogroup,2014,Mazda
...,...,...,...,...,...,...,...,...,...,...,...
133,Grand,https://www.cheapcarco.com.au/used-vehicles-ro...,11999,"200,001+",Automatic,Petrol,Wagon,20251108,cheapcarco,2012,Suzuki
134,HS,https://www.cheapcarco.com.au/used-vehicles-ro...,17999,"60,001 - 80,000",Automatic,Petrol,Wagon,20251108,cheapcarco,2019,MG
135,i30,https://www.cheapcarco.com.au/used-vehicles-ro...,7999,"160,001 - 180,000",Automatic,Petrol,Wagon,20251108,cheapcarco,2011,Hyundai
136,i30,https://www.cheapcarco.com.au/used-vehicles-ro...,9999,"140,001 - 160,000",Automatic,Petrol,Wagon,20251108,cheapcarco,2013,Hyundai


In [60]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1957 entries, 0 to 137
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   name           1852 non-null   object
 1   link_product   1957 non-null   object
 2   price          1957 non-null   int64 
 3   odometer       1957 non-null   object
 4   transmissions  1957 non-null   object
 5   fuel_type      1957 non-null   object
 6   car_type       1957 non-null   object
 7   update_date    1957 non-null   int64 
 8   dealership     1957 non-null   object
 9   year_made      1957 non-null   int64 
 10  brand          1957 non-null   object
dtypes: int64(3), object(8)
memory usage: 183.5+ KB


In [61]:
def track_sold (df, sku_col = 'link_product', date_col = 'update_date'):
    df[date_col]= pd.to_datetime(df[date_col], format='%Y%m%d', errors='coerce')
    max_date = df[date_col].max()

    last_seen = (df.groupby(sku_col)[date_col].max().rename('sold_date'))
    sold_dates = last_seen[last_seen < max_date] + timedelta(days=1)
    df['sold_date'] = df[sku_col].map(sold_dates)

    return df

In [62]:
def track_import (df, sku_col = 'link_product', date_col = 'update_date'):
    df[date_col]= pd.to_datetime(df[date_col], format='%Y%m%d', errors='coerce')
    first_seen = (df.groupby(sku_col)[date_col].min().rename('import_date'))
    df['import_date'] = df[sku_col].map(first_seen)

    return df

In [ ]:
df = track_sold(df)
df = track_import(df)
df['import_date'] = df['import_date'].replace('2025-11-05',None)


df

,name,link_product,price,odometer,transmissions,fuel_type,car_type,update_date,dealership,year_made,brand,sold_date,import_date
0,Mustang,/used-cars/for-sale/ford/mustang/2017/fm-my17/...,33990,"180,001 - 200,000",Automatic,Petrol,Coupe,2025-11-05,brisbaneautogroup,2017,Ford,2025-11-08,NaT
1,Fiesta,/used-cars/for-sale/ford/fiesta/2015/ambiente/...,6990,"180,001 - 200,000",Manual,Petrol,Hatchback,2025-11-05,brisbaneautogroup,2015,Ford,2025-11-06,NaT
2,2,/used-cars/for-sale/mazda/2/2011/neo/de-series...,7490,"160,001 - 180,000",Automatic,Petrol,Hatchback,2025-11-05,brisbaneautogroup,2011,Mazda,NaT,NaT
3,Micra,/used-cars/for-sale/nissan/micra/2016/st/k13-m...,7990,"120,001 - 140,000",Automatic,Petrol,Hatchback,2025-11-05,brisbaneautogroup,2016,Nissan,2025-11-08,NaT
4,3,/used-cars/for-sale/mazda/3/2014/sp25-astina/b...,9990,"140,001 - 160,000",Automatic,Petrol,Hatchback,2025-11-05,brisbaneautogroup,2014,Mazda,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,Grand,https://www.cheapcarco.com.au/used-vehicles-ro...,11999,"200,001+",Automatic,Petrol,Wagon,2025-11-08,cheapcarco,2012,Suzuki,NaT,NaT
134,HS,https://www.cheapcarco.com.au/used-vehicles-ro...,17999,"60,001 - 80,000",Automatic,Petrol,Wagon,2025-11-08,cheapcarco,2019,MG,NaT,NaT
135,i30,https://www.cheapcarco.com.au/used-vehicles-ro...,7999,"160,001 - 180,000",Automatic,Petrol,Wagon,2025-11-08,cheapcarco,2011,Hyundai,NaT,NaT
136,i30,https://www.cheapcarco.com.au/used-vehicles-ro...,9999,"140,001 - 160,000",Automatic,Petrol,Wagon,2025-11-08,cheapcarco,2013,Hyundai,NaT,2025-11-06


In [64]:
output = Path ('output')
output.mkdir(parents=True, exist_ok=True)
filename= f'{ts}_data.csv'
file_path = output / filename
df.to_csv(file_path, index=False, encoding="utf-8-sig")